In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.5267725239973515 
Enpoint: people/475627 | Time: 0.4826185750134755 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.4654022110044025 
PROJECTS
Enpoint: projects | Time: 0.5756250320118852 
ROLES
Enpoint: roles | Time: 0.6232219569792505 
PEOPLE
Enpoint: people | Time: 0.5926475859887432 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.4724513879918959 
MILESTONES
Enpoint: milestones | Time: 0.4530250550014898 
ASSIGNMENTS
Enpoint: assignments | Time: 0.8466162160038948 
DONE! (5.1s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

In [5]:
names = list(fc.people.full_name)
names.remove('Giovanni Colavizza')
names.remove('Miguel Morin')
names.remove('Mohammed Ali Al-Badri')
names

['Oliver Strickson',
 'Angus Williams',
 'David Beavan',
 'Evelina Gabasova',
 'James Geddes',
 'James Hetherington',
 'Louise Bowler',
 "Martin O'Reilly",
 'May Yong',
 'Nick Barlow',
 'Radka Jersakova',
 'Timothy Hobson',
 'Tomas Lazauskas',
 'Camila Rangel Smith',
 'Sarah Gibson',
 'James Robinson',
 'Eric Daub',
 'Jim Madge',
 'Amber Raza',
 'Kasra Hosseini',
 'Oscar Giles',
 'Joel Dearden',
 'Amaani Hoddoon',
 'Jack Roberts',
 'Oliver Forrest',
 'Gabriel Hanganu',
 'Ed Chalstrey',
 'Ashwini Venkatasubramaniam',
 'Roly Perera']

Find a team member's availability for a date range
---

In [7]:
def get_person_availability(name, start_date, end_date):
    """Get the mean of a person's FTE proportion available for the start to end datetime objects"""
    try:
        person_id = fc.get_person_id(name)
    except:
        return 0.0
    peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    availability_range = peopledf[person_id]
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [8]:
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
get_person_availability("Ed Chalstrey", start_date, end_date)

0.77

Create queries with GraphQL to get the emojis for each GitHub issue
===

https://developer.github.com/v4/guides/forming-calls/

In [10]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [11]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

In [46]:
def get_preference_data(fc):
    """Get each team members preference emoji for all projects with a GitHub issue"""
    issues = fc.projects["GitHub"].dropna()  # Get list of GitHub issues for projects
    gid_mapping = {  # People without their full names on github.
     'myyong': 'May Yong',
     'nbarlowATI': 'Nick Barlow',
     'thobson88': 'Timothy Hobson',
     'miguelmorin': 'Miguel Morin',
     'OscartGiles': 'Oscar Giles',
     'AshwiniKV': 'Ashwini Venkatasubramaniam',
    }
    emoji_mapping = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}
    names = list(fc.people.full_name)
    preference_data = {
        "Person": names
    }
    for issue_num, project_id in zip(issues, issues.index):
        modified_query = query.replace("X", str(issue_num))
        result = run_query(modified_query)  # Execute the query
#         if query_result_contains_emojis(result)  # Could do this if we only want issues with emojis
        emojis = []
        for name in names: 
            emoji_name = None
            for reaction in result['data']['repository']['issue']['reactionGroups']:
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] == name:
                        emoji_name = reaction['content']
                        if not emoji_name:
                            if gid_mapping[edge['node']['login']] == name:
                                emoji_name = reaction['content']
            if emoji_name:
                emoji = emoji_mapping[emoji_name]
            else:
                emoji = "❓"
            emojis.append(emoji)
        preference_data[fc.get_project_name(project_id)] = emojis
    return preference_data

In [47]:
preference_data = get_preference_data(fc)

In [48]:
preference_data_df = pd.DataFrame(preference_data).set_index('Person')
preference_data_df

,Sargasso (Intel Sparse),DetectorChecker,HSBC EDS,AI for City Planning,Reproducible Research,PDQ,Crypto Federated Learning (SHEEP),The Turing Way,Fitbit classification,Parallel Monte Carlo,...,NHS Scotland Cancer Waiting Times,DFTG Commons scoping,DFTG Commons WP 3.4/3.5,CHANCE,Scaleable Monte Carlo,Wrattler 2019-2020,PDMP,Urban Agriculture,Interpretable models,Data science benchmarking- follow on
Person,,,,,,,,,,,,,,,,,,,,,
Oliver Strickson,❓,❓,❓,❓,❓,❓,❓,❌,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Angus Williams,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
David Beavan,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Evelina Gabasova,❓,❓,❓,❓,❓,❓,❓,✅,👍,👍,...,❓,❌,❌,👍,❌,❓,❓,❓,❓,❓
Giovanni Colavizza,❓,❓,❓,❓,❓,❓,❓,👍,❌,❌,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
James Geddes,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,👍,❓,❓,❓,❓,❓
James Hetherington,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓
Louise Bowler,❓,❓,❓,❓,❓,❓,❓,✅,❌,❌,...,❓,❌,❌,👍,👍,❓,❓,❓,❓,❓
Martin O'Reilly,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓,...,❓,❓,❓,❓,❓,❓,❓,❓,❓,❓


In [49]:
preference_data_df["CHANCE"]["Ed Chalstrey"]

'✅'

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [19]:
def get_preferences(fc, first_date=False, last_date=False, person=False, project=False, positive_only=False):
    issues = fc.projects["GitHub"].dropna().values  # Get list of GitHub issues for projects
    gid_mapping = {  # People without their full names on github.
     'myyong': 'May Yong',
     'nbarlowATI': 'Nick Barlow',
     'thobson88': 'Timothy Hobson',
     'miguelmorin': 'Miguel Morin',
     'OscartGiles': 'Oscar Giles',
     'AshwiniKV': 'Ashwini Venkatasubramaniam',
    }
    resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
    if person:
        names = [person]
    else:
        names = list(fc.people.full_name)
    data = {
        "Person": names
    }
    for project_id in resreqdf:
        for resreq in resreqdf[project_id]:
            if resreq > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
                issue_num = fc.projects.loc[project_id, "GitHub"]
                if issue_num in issues:
                    start_date = fc.projects.loc[project_id, "start_date"]
                    end_date = fc.projects.loc[project_id, "end_date"]
                    first_resreq_date = resreqdf.index[resreqdf[project_id] != 0][0].strftime("%Y-%m-%d")
                    project_title = fc.projects.loc[project_id, "name"]
                    modified_query = query.replace("X", str(issue_num))
                    result = run_query(modified_query)  # Execute the query
                    emoji_data = []
                    for name in names:                 
                        person_availability = get_person_availability(name, first_resreq_date, end_date)
                        percentage_availability = round((person_availability / resreq) * 100, 2)
                        emoji_name = None
                        for reaction in result['data']['repository']['issue']['reactionGroups']:
                            for edge in reaction['users']['edges']:
                                if edge['node']['name'] == name:
                                    emoji_name = reaction['content']
                                    if not emoji_name:
                                        if gid_mapping[edge['node']['login']] == name:
                                            emoji_name = reaction['content']
                        if emoji_name:
                            emoji = emojis[emoji_name]
                        else:
                            emoji = "❓"
                        emoji_data.append(emoji + " " + str(percentage_availability) + "% (" + str(person_availability) + ")")
                    data[project_title] = emoji_data
                break
    return pd.DataFrame(data)

All projects with team members preferences and their average availability between the start of project resource requirement and project end date
----

In [20]:
get_preferences(fc)

,Person,Living With Machines,AIDA-Lloyds,Probabilistic FEM,Safe Haven,UQM^3,Auto DL,All in One Cancer imaging optimisation,Fusion modelling- IDEA,Uncertainty in Government Modelling,...,Logics for DS,Medevac Decision Support,NHS Medication Safety,UCLH Cancer Waiting Times,NHS Scotland Cancer Waiting Times,DFTG Commons scoping,DFTG Commons WP 3.4/3.5,CHANCE,Scaleable Monte Carlo,Wrattler 2019-2020
0,Oliver Strickson,❓ 70.0% (0.7),❓ 0.0% (0.0),❓ 24.0% (0.12),❓ 42.0% (0.42),👍 58.0% (0.29),❌ 0.0% (0.0),❓ 67.0% (0.67),❓ 67.0% (0.67),❓ 34.0% (0.17),...,❌ 168.0% (0.42),❓ 17.0% (0.17),❓ 0.0% (0.0),❓ 17.0% (0.17),❓ 50.0% (0.5),❓ 0.0% (0.0),❓ 39.5% (0.79),❓ 42.0% (0.21),❓ 100.0% (1.0),❓ 17.0% (0.17)
1,Angus Williams,❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),...,❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0)
2,David Beavan,❓ 41.0% (0.41),❓ 0.0% (0.0),❓ 24.0% (0.12),❓ 29.0% (0.29),❓ 58.0% (0.29),❓ 0.0% (0.0),❓ 42.0% (0.42),❓ 42.0% (0.42),❓ 34.0% (0.17),...,❓ 128.0% (0.32),❓ 17.0% (0.17),❓ 0.0% (0.0),❓ 17.0% (0.17),❓ 50.0% (0.5),❓ 0.0% (0.0),❓ 25.0% (0.5),❓ 42.0% (0.21),❓ 50.0% (0.5),❓ 17.0% (0.17)
3,Evelina Gabasova,👍 41.0% (0.41),👍 0.0% (0.0),❌ 24.0% (0.12),❓ 29.0% (0.29),❌ 58.0% (0.29),👍 0.0% (0.0),👍 42.0% (0.42),❌ 42.0% (0.42),👍 34.0% (0.17),...,👍 128.0% (0.32),❓ 17.0% (0.17),👍 0.0% (0.0),❓ 17.0% (0.17),❓ 50.0% (0.5),❌ 0.0% (0.0),❌ 25.0% (0.5),👍 42.0% (0.21),❌ 50.0% (0.5),❓ 17.0% (0.17)
4,Giovanni Colavizza,✅ 0.0% (0.0),👍 0.0% (0.0),❓ 0.0% (0.0),❓ 3.0% (0.03),❌ 0.0% (0.0),👍 0.0% (0.0),❓ 0.0% (0.0),❌ 0.0% (0.0),❌ 0.0% (0.0),...,👍 4.0% (0.01),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 4.0% (0.02),❓ 0.0% (0.0),❓ 0.0% (0.0)
5,James Geddes,❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),...,❓ 0.0% (0.0),❓ 0.0% (0.0),👍 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),👍 0.0% (0.0),❓ 0.0% (0.0)
6,James Hetherington,❓ 10.0% (0.1),❓ 25.0% (0.1),✅ 20.0% (0.1),❓ 9.0% (0.09),❓ 20.0% (0.1),❓ 20.0% (0.1),❓ 10.0% (0.1),❓ 10.0% (0.1),❓ 20.0% (0.1),...,❓ 40.0% (0.1),❓ 10.0% (0.1),❓ 20.0% (0.1),❓ 10.0% (0.1),❓ 10.0% (0.1),❓ 10.0% (0.1),❓ 5.0% (0.1),❓ 20.0% (0.1),❓ 10.0% (0.1),❓ 10.0% (0.1)
7,Louise Bowler,👍 67.0% (0.67),👍 100.0% (0.4),❌ 88.0% (0.44),❓ 40.0% (0.4),👍 92.0% (0.46),👍 84.0% (0.42),👍 58.0% (0.58),❓ 58.0% (0.58),❌ 88.0% (0.44),...,❌ 188.0% (0.47),👍 42.0% (0.42),✅ 80.0% (0.4),❓ 42.0% (0.42),❓ 50.0% (0.5),❌ 38.0% (0.38),❌ 25.0% (0.5),👍 92.0% (0.46),👍 75.0% (0.75),❓ 44.0% (0.44)
8,Martin O'Reilly,❓ 18.0% (0.18),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 10.0% (0.1),❓ 10.0% (0.1),❓ 0.0% (0.0),...,❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 0.0% (0.0),❓ 25.0% (0.25),❓ 0.0% (0.0)
9,May Yong,❓ 80.0% (0.8),❓ 50.0% (0.2),❓ 50.0% (0.25),❓ 56.0% (0.56),❓ 106.0% (0.53),❓ 32.0% (0.16),❓ 81.0% (0.81),❓ 81.0% (0.81),❓ 56.0% (0.28),...,❓ 240.0% (0.6),❓ 33.0% (0.33),❓ 20.0% (0.1),❓ 33.0% (0.33),❓ 75.0% (0.75),❓ 24.0% (0.24),❓ 50.0% (1.0),❓ 74.0% (0.37),❓ 100.0% (1.0),❓ 28.0% (0.28)


In [21]:
get_preferences(fc, person="Ed Chalstrey")

,Person,Living With Machines,AIDA-Lloyds,Probabilistic FEM,Safe Haven,UQM^3,Auto DL,All in One Cancer imaging optimisation,Fusion modelling- IDEA,Uncertainty in Government Modelling,...,Logics for DS,Medevac Decision Support,NHS Medication Safety,UCLH Cancer Waiting Times,NHS Scotland Cancer Waiting Times,DFTG Commons scoping,DFTG Commons WP 3.4/3.5,CHANCE,Scaleable Monte Carlo,Wrattler 2019-2020
0,Ed Chalstrey,❓ 39.0% (0.39),❓ 250.0% (1.0),❓ 200.0% (1.0),❓ 52.0% (0.52),❓ 186.0% (0.93),❌ 200.0% (1.0),✅ 46.0% (0.46),❓ 46.0% (0.46),❓ 200.0% (1.0),...,❌ 284.0% (0.71),❌ 100.0% (1.0),👍 200.0% (1.0),❓ 100.0% (1.0),❓ 100.0% (1.0),👍 100.0% (1.0),❓ 16.5% (0.33),✅ 192.0% (0.96),👍 0.0% (0.0),❓ 100.0% (1.0)
